## **Library and Module Imports**  
  
The following code block imports all the essential libraries and frameworks required for the Clinical Intelligence System. These libraries support environment configuration, AI model integration, vector storage, data loading, evaluation, and performance metrics.  
  
### **Key Imports and Their Purpose**  
  
1. **Core AI and Environment Setup**  
   - `import openai` – Provides access to OpenAI's API for natural language processing and model integration.  
   - `from dotenv import load_dotenv` – Loads environment variables from a `.env` file to securely store API keys and configuration values.  
  
2. **Vector Storage and Embeddings**  
   - `from langchain.vectorstores import Chroma` – Manages vector databases for semantic search and retrieval.  
   - `from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI` – Integrates Azure-hosted OpenAI models for embeddings and chat-based language models.  
  
3. **Document Handling**  
   - `from langchain.schema import Document` – Defines structured document objects for processing.  
   - `from langchain_community.document_loaders.csv_loader import CSVLoader` – Loads CSV files into a document structure for further processing.  
  
4. **Utilities and Data Structures**  
   - `from typing import List, Tuple, Dict` – Provides type hints for function parameters and return values.  
   - `import numpy as np` – Supports numerical operations, arrays, and mathematical computations.  
   - `import time` – Enables time tracking and performance measurement.  
  
5. **Retry Mechanisms**  
   - `from tenacity import retry, stop_after_attempt, wait_random_exponential` – Implements robust retry logic to handle API timeouts and transient failures.  
  
6. **Evaluation Framework**  
   - `from deepeval.models.base_model import DeepEvalBaseLLM` – Base class for evaluating large language models.  
   - `from deepeval.test_case import LLMTestCase, LLMTestCaseParams` – Defines structured test cases for model evaluation.  
   - `from deepeval import evaluate as deepeval_evaluate` – Runs evaluation processes for AI model outputs.  
  
7. **Evaluation Metrics**  
   - `from deepeval.metrics import (ContextualPrecisionMetric, ContextualRecallMetric, ContextualRelevancyMetric, AnswerRelevancyMetric, FaithfulnessMetric, HallucinationMetric)`    
     - **ContextualPrecisionMetric** – Measures the accuracy of retrieved information within the given context.    
     - **ContextualRecallMetric** – Measures how much relevant context is retrieved.    
     - **ContextualRelevancyMetric** – Evaluates the contextual fit of retrieved information.    
     - **AnswerRelevancyMetric** – Assesses how relevant the generated answer is to the question.    
     - **FaithfulnessMetric** – Ensures the answer is factually grounded in the source material.    
     - **HallucinationMetric** – Detects fabricated or unsupported information in responses.  
  
---  
  
**Summary:**    
This set of imports lays the groundwork for:  
- **NLP processing** (OpenAI, Azure OpenAI, LangChain)  
- **Data management** (Chroma DB, CSV loaders, Document schema)  
- **Robustness** (retry mechanisms)  
- **Evaluation** (DeepEval metrics and test cases)    
These tools collectively enable the system to process clinical queries, retrieve relevant data, generate responses, and evaluate their quality.  

In [ ]:
import openai
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain.schema import Document
from langchain_community.document_loaders.csv_loader import CSVLoader

from typing import List, Tuple, Dict
import numpy as np  
import time

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

from deepeval.models.base_model import DeepEvalBaseLLM
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval import evaluate as deepeval_evaluate
from deepeval.metrics import (
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric,
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    HallucinationMetric,
)

### Create Model Client and Set Up Authentication

The following code initializes the UAIS environment to establish a secure connection with the Azure OpenAI service. It handles authentication by retrieving the necessary access token and configures the embedding function to generate vector representations for input text. This setup enables downstream tasks such as semantic search, similarity comparison, and other embedding-based applications.

| Requirement           | Description                                                        |  
|-----------------------|--------------------------------------------------------------------|  
| Large Language Models (LLM) | OpenAI LLM API (`gpt-4o-mini_2024-07-18`)                           |  
| Embedding Models      | Preferred embedding model is `text-embedding-3-small_1`            |  

In [ ]:
# Authentication:
import httpx

auth = "https://api.com/oauth2/token"
client_id = dbutils.secrets.get(scope = "AIML", key = "client_id")
client_secret = dbutils.secrets.get(scope = "AIML", key = "client_secret")
scope = "https://api.com/.default"
grant_type = "client_credentials"
async with httpx.AsyncClient() as client:
    body = {
        "grant_type": grant_type,
        "scope": scope,
        "client_id": client_id,
        "client_secret": client_secret,
    }
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    resp = await client.post(auth, headers=headers, data=body, timeout=120)
    token = resp.json()["access_token"]


load_dotenv("./Data/vars.env")

AZURE_OPENAI_ENDPOINT = os.environ["MODEL_ENDPOINT"]
OPENAI_API_VERSION = os.environ["API_VERSION"]
CHAT_DEPLOYMENT_NAME = os.environ["CHAT_MODEL_NAME"]
PROJECT_ID = os.environ["PROJECT_ID"]
EMBEDDINGS_DEPLOYMENT_NAME = os.environ["EMBEDDINGS_MODEL_NAME"]

chat_client = openai.AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=OPENAI_API_VERSION,
    azure_deployment=CHAT_DEPLOYMENT_NAME,
    azure_ad_token=token,
    default_headers={
        "projectId": PROJECT_ID
    }
)

embeddings_client = openai.AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=OPENAI_API_VERSION,
    azure_deployment=EMBEDDINGS_DEPLOYMENT_NAME,
    azure_ad_token=token,
    default_headers={ 
        "projectId": PROJECT_ID
    }
)

## Azure OpenAI Model & Embeddings Setup  
  
This section initializes the Azure OpenAI resources required for the RAG pipeline:  
  
- **`AzureChatOpenAI`** – Configures a chat-based LLM endpoint using Azure OpenAI, enabling conversational interactions and contextual responses.  
- **`AzureOpenAIEmbeddings`** – Sets up an embedding model to convert text into high-dimensional vectors for semantic search and retrieval.  
- Both components share:  
  - The same **API version** and **Azure endpoint**.  
  - **Azure AD token authentication** for secure access.  
  - Custom **`projectId`** in request headers for project-level tracking.  

In [ ]:
chat_model = AzureChatOpenAI(
    openai_api_version=OPENAI_API_VERSION,
    azure_deployment=CHAT_DEPLOYMENT_NAME,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_ad_token=token,
    default_headers={"projectId": PROJECT_ID},
)

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=EMBEDDINGS_DEPLOYMENT_NAME,
    api_version=OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_ad_token=token,
    default_headers={
        "projectId": PROJECT_ID
    }
)

### Tiktoken Cache Configuration
 
> This code sets up a custom cache directory for Tiktoken by defining `TIKTOKEN_CACHE_DIR` as an environment variable.  
> Local caching of tokenization results enhances performance by avoiding repeated computation during recurring embedding or tokenization tasks.

In [ ]:
tiktoken_cache_dir = os.path.abspath("./.setup/tiktoken_cache/")
os.environ["TIKTOKEN_CACHE_DIR"] = tiktoken_cache_dir

# we have to disable telemetry to use ChromaDB
# See here for more information: https://docs.trychroma.com/docs/overview/telemetry
os.environ["ANONYMIZED_TELEMETRY"]="False"

## DocumentProcessor Class  
  
The `DocumentProcessor` class leverages **LangChain's CSVLoader** to efficiently ingest CSV datasets and convert them into LangChain `Document` objects. This design ensures:  
  
- **Built-in CSV parsing** with automatic conversion to Document objects.    
- **Native LangChain document structure** for smooth integration into RAG pipelines.    
- **Standardized metadata extraction** adhering to LangChain conventions.    
- **Direct compatibility** with LangChain’s ecosystem of loaders, retrievers, and vector stores.  


In [ ]:
class DocumentProcessor:
    """
    Streamlined Document Processing Engine for RAG pipeline using LangChain CSVLoader.
    """
    
    def __init__(self):
        """
        Initialize the DocumentProcessor.
        """
        self.langchain_docs = []  # List of LangChain Document objects for RAG
        
    def load_csv_with_langchain(self, csv_path: str) -> List[Document]:
        """
        Load CSV data using LangChain CSVLoader.
        
        Args:
            csv_path (str): Path to the CSV file to load
            
        Returns:
            List[Document]: List of LangChain Document objects
        """
        # Configure CSVLoader with our dataset structure
        loader = CSVLoader(
            file_path=csv_path,
            source_column="document_url",  # Use document_url as source
            metadata_columns=["document_id", "document_url"],  # Include these in metadata
            content_columns=["context"]  # Use context as main content
        )
        
        # Load documents using LangChain
        documents = loader.load()
        
        print(f"Successfully loaded {len(documents)} document chunks from CSV using LangChain CSVLoader.")
        return documents
        
    def load_dataset(self, dataset_path: str) -> List[Document]:
        """
        Load documents from CSV using LangChain CSVLoader and return Documents ready for vector storage.
        
        Args:
            dataset_path (str): Path to the CSV dataset file
            
        Returns:
            List[Document]: List of LangChain Document objects
            
        Raises:
            FileNotFoundError: If the dataset file is not found
            Exception: If there's an error loading the CSV file
        """
        try:
            # Check if file exists
            if not os.path.exists(dataset_path):
                raise FileNotFoundError(f"Dataset file not found: {dataset_path}")
            
            print(f"📁 Loading dataset from: {dataset_path}")
            
            # Use LangChain CSVLoader 
            self.langchain_docs = self.load_csv_with_langchain(dataset_path)
            
            print(f"✅ Dataset loaded successfully using LangChain CSVLoader")
            print(f"📚 Loaded {len(self.langchain_docs)} LangChain Document objects")
            
            # Display first document for verification
            if self.langchain_docs:
                print(f"\n🔍 Sample LangChain Document:")
                sample_doc = self.langchain_docs[0]
                print(f"   Content: {sample_doc.page_content[:100]}...")
                print(f"   Metadata: {sample_doc.metadata}")
            
            return self.langchain_docs
            
        except FileNotFoundError as e:
            print(f"❌ File not found: {e}")
            raise
        except Exception as e:
            print(f"❌ Error loading dataset with CSVLoader: {e}")
            raise
    
    def get_langchain_documents(self) -> List[Document]:
        """
        Get the loaded LangChain Documents ready for vector storage.
        
        Returns:
            List[Document]: List of LangChain Document objects
        """
        return self.langchain_docs
    
    def get_document_count(self) -> int:
        """Get the number of loaded documents."""
        return len(self.langchain_docs)
    
    def display_dataset_info(self) -> None:
        """Display essential information about the loaded dataset for RAG."""
        if not self.langchain_docs:
            print("⚠️ No documents loaded. Call load_dataset() first.")
            return
        
        print(f"\n📊 === Dataset Information for RAG Pipeline ===")
        print(f"📚 Total documents: {len(self.langchain_docs)}")
        
        if self.langchain_docs:
            # Show sample LangChain document structure
            print(f"\n📄 === Sample LangChain Document ===")
            sample_doc = self.langchain_docs[0]
            print(f"page_content: {sample_doc.page_content[:200]}...")
            print(f"metadata: {sample_doc.metadata}")
            
            # Show content length statistics
            content_lengths = [len(doc.page_content) for doc in self.langchain_docs]
            print(f"\n📊 === Content Statistics ===")
            print(f"Average content length: {sum(content_lengths) / len(content_lengths):.0f} characters")
            print(f"Shortest document: {min(content_lengths)} characters")
            print(f"Longest document: {max(content_lengths)} characters")

print("✅ LangChain CSVLoader-based DocumentProcessor defined successfully!")

## Vector Store Creation with Check-and-Reuse Logic  
  
This section defines a **smart vector store creation pipeline** for efficient embedding management.  
  
- **Purpose:**    
  - Checks if a persisted ChromaDB vector store already exists.    
  - **If found:** Loads and reuses existing embeddings without regeneration.    
  - **If not found:** Creates a new vector store from the loaded documents, generates embeddings, and persists the store.  
  
- **Key Features:**    
  - Uses `DocumentProcessor` to retrieve preloaded LangChain documents.    
  - Embeddings are created via the configured Azure OpenAI embedding model.    
  - Ensures persistence in a specified directory for later reuse.    
  - Includes diagnostic statistics (vector count, embedding dimensions, range, and norms).    
  - Built with retry logic to handle transient failures during creation or loading.  
  
- **Benefit:**    
  This approach saves computation time and cost by **avoiding redundant embedding generation** while still supporting full regeneration when required.  

In [ ]:
# Vector Store Creation Pipeline with Check-and-Reuse Logic
@retry(wait=wait_random_exponential(min=45, max=120), stop=stop_after_attempt(6))
def create_vector_store(processor: DocumentProcessor, collection_name: str = "clinical_intelligence", persist_directory: str = "./Data/clinical_rag.db"):
    """
    Create embeddings and store in ChromaDB vector store with intelligent reuse.
    
    Purpose:
    - Checks whether a vector store already exists in the specified directory
    - If it does, loads and reuses the existing vector store
    - If it doesn't, creates a new vector store from the provided documents and persists it
    
    Args:
        processor (DocumentProcessor): Loaded document processor
        collection_name (str): Name for the ChromaDB collection
        persist_directory (str): Directory path for ChromaDB persistence
        
    Returns:
        Chroma: ChromaDB vector store with embedded documents
    """
    print(f"\n🚀 === Clinical Intelligence Pipeline - Step 2: Vector Store Creation ===")
    print(f"📂 Checking persist directory: {persist_directory}")
    print(f"🗂️ Collection name: {collection_name}")
    
    try:
        # Check if vector store already exists
        if os.path.exists(persist_directory) and os.listdir(persist_directory):
            print(f"📋 Existing vector store found in: {persist_directory}")
            print(f"🔄 Loading existing vector store...")
            
            # Load existing vector store
            vector_store = Chroma(
                collection_name=collection_name,
                embedding_function=embeddings,
                persist_directory=persist_directory
            )
            
            print(f"✅ Existing vector store loaded successfully!")
            print(f"📚 Found {vector_store._collection.count()} existing vectors")
            print(f"🔄 Reusing existing embeddings (no regeneration needed)")
            
        else:
            print(f"📭 No existing vector store found")
            print(f"🆕 Creating new vector store from documents...")
            print(f"📊 Processing {processor.get_document_count()} documents for embedding...")
            
            # Get documents from processor
            documents = processor.get_langchain_documents()
            
            if not documents:
                raise ValueError("No documents loaded. Run clinical_intelligence_pipeline first.")
            
            # Create new vector store with embeddings
            print(f"🔍 Creating embeddings using text-embedding-3-small model...")
            print(f"💾 Storing vectors in ChromaDB collection: '{collection_name}'")
            
            # Create vector store - this will automatically generate embeddings and store them
            vector_store = Chroma.from_documents(
                documents=documents,
                embedding=embeddings,
                collection_name=collection_name,
                persist_directory=persist_directory
            )
            
            print(f"✅ New vector store created successfully!")
            print(f"📚 Embedded and stored {len(documents)} documents")
            print(f"💾 Persisted to: {persist_directory}")
        
        # Display common statistics
        print(f"\n📊 === Vector Store Statistics ===")
        print(f"Total vectors: {vector_store._collection.count()}")
        print(f"Collection name: {collection_name}")
        print(f"Persist directory: {persist_directory}")
        print(f"Embedding model: text-embedding-3-small")
        
        # Test embedding dimension using actual document content from CSV
        try:
            # Use first document's content for testing embedding
            documents = processor.get_langchain_documents()
            if documents and len(documents) > 0:
                # Take first 100 characters from first document for testing
                test_content = documents[0].page_content[:100]
                test_embedding = embeddings.embed_query(test_content)
                print(f"Vector dimension: {len(test_embedding)}")
                print(f"Test content: '{test_content[:50]}...'")
                print(f"Sample embedding values (first 10): {test_embedding[:10]}")
                print(f"Embedding range: [{min(test_embedding):.6f}, {max(test_embedding):.6f}]")
                print(f"Embedding norm: {sum(x*x for x in test_embedding)**0.5:.6f}")
            else:
                print(f"⚠️ No documents available for embedding test")
                print(f"Vector dimension: Unknown (no test performed)")
        except Exception as e:
            print(f"Vector dimension: Available in collection (test failed: {e})")
        
        return vector_store
        
    except Exception as e:
        print(f"❌ Error with vector store: {e}")
        raise

print("✅ Smart vector store creation function defined successfully!")
print("🔄 Supports both new creation and existing vector store reuse!")

## Complete Clinical Intelligence Pipeline  
  
This function orchestrates the **full data-to-vector pipeline** for the Clinical Intelligence system, combining both document ingestion and vectorization.  
  
- **Purpose:**    
  - Streamline the process of loading CSV-based clinical data into LangChain `Document` objects.    
  - Automatically embed documents into a ChromaDB vector store for semantic search and retrieval.  
  
- **Workflow Steps:**    
  1. **Document Processing:**    
     - Instantiates a `DocumentProcessor`.    
     - Loads the dataset via LangChain's `CSVLoader`.    
     - Prepares documents for downstream vectorization.  
  2. **Vector Store Creation:**    
     - Invokes the smart `create_vector_store()` function.    
     - Either reuses an existing ChromaDB store or generates embeddings for new documents.    
     - Persists the vector store for future queries.  
  
- **Output:**    
  Returns a tuple of `(DocumentProcessor, Chroma)` for immediate use in query workflows.  
  
- **Benefit:**    
  Offers a **one-command execution** for setting up the RAG-ready clinical intelligence environment, ensuring consistency between document loading and embedding stages.  

In [ ]:
# Complete Clinical Intelligence Pipeline - Data Loading + Vectorization
def complete_clinical_intelligence_pipeline(dataset_path: str, persist_directory: str, collection_name: str = "clinical_intelligence"):
    """
    Complete Clinical Intelligence Pipeline that handles both document processing and vector store creation.
    
    Args:
        dataset_path (str): Path to the CSV dataset file
        persist_directory (str): Directory path for ChromaDB persistence
        collection_name (str): Name for the ChromaDB collection
        
    Returns:
        tuple: (DocumentProcessor, Chroma) - processor and vector store
    """
    print(f"🏥 === Complete Clinical Intelligence Pipeline ===")
    print(f"📂 Dataset path: {dataset_path}")
    print(f"💾 Persist directory: {persist_directory}")
    print(f"🗂️ Collection name: {collection_name}")
    
    try:
        # Step 1: Document Processing
        print(f"\n{'='*60}")
        print(f"🚀 === Step 1: Document Processing ===")
        print(f"📂 Dataset path: {dataset_path}")
        
        # Initialize the document processor
        processor = DocumentProcessor()
        
        # Load the dataset using LangChain CSVLoader
        langchain_documents = processor.load_dataset(dataset_path)
        print(f"\n🎉 Successfully loaded {processor.get_document_count()} LangChain Documents using CSVLoader!")
        print(f"✅ Documents are ready for vector storage and embeddings")
        print(f"🔄 Using same loading pattern as PDF documents for consistency")
        
        # Step 2: Vector Store Creation
        print(f"\n{'='*60}")
        print(f"🚀 === Step 2: Vector Store Creation ===")
        vector_store = create_vector_store(
            processor, 
            collection_name=collection_name, 
            persist_directory=persist_directory
        )
        
        print(f"\n🎉 === Complete Clinical Intelligence Pipeline Finished ===")
        print(f"✅ Step 1: Document processing completed - {processor.get_document_count()} documents loaded")
        print(f"✅ Step 2: Vector store creation completed - {vector_store._collection.count()} vectors stored")
        print(f"🚀 Clinical Intelligence system ready for queries!")
        
        return processor, vector_store
        
    except Exception as e:
        print(f"❌ Error in complete pipeline: {e}")
        raise

In [ ]:
# Define configuration variables
dataset_path = "./Data/capstone1_rag_dataset.csv"
persist_directory = "./Data/clinical_rag.db"
collection_name = "clinical_intelligence_v1"

# Execute Complete Pipeline in One Call
processor, vector_store = complete_clinical_intelligence_pipeline(
    dataset_path=dataset_path,
    persist_directory=persist_directory,
    collection_name=collection_name
)


## Retrieval Strategy Exploration  
  
In this section, we evaluate multiple retrieval strategies to identify the most effective approach for clinical document retrieval. Each method balances precision, recall, and relevance differently, aiming to ensure that retrieved documents are both accurate and contextually aligned with the query.  
  
### 🎯 Retrieval Strategies to Compare  
  
- **Semantic Search**    
  Utilizes pure vector similarity to retrieve documents that are semantically closest to the query, enabling context-aware matching beyond exact keywords.  
  
- **Semantic Search with Threshold Filtering**    
  Builds on semantic similarity but applies a relevance score cutoff, ensuring only highly relevant documents are returned and reducing noise.  
  
- **Hybrid Search**    
  Combines keyword-based retrieval with semantic similarity to capture both exact matches and contextual relevance, improving coverage for diverse query types.  
  
- **Re-ranking based on Relevance Scores**    
  Retrieves a broader set of candidates, then applies a secondary scoring mechanism to reorder results for optimal relevance and user satisfaction.  

### Strategy 1: Pure Semantic Search  
  
**Purpose:**    
This approach retrieves documents purely based on vector similarity, leveraging semantic embeddings to capture contextual meaning rather than relying on exact keyword matches. It is designed to surface conceptually relevant results even when terminology differs between the query and the source text.  
  
**Evaluation Summary:**    
- **Query:** "What are the early signs and available therapies for high blood sugar caused by lack of insulin?"    
- **Results Found:** 5 documents    
- **Observation:** Returned top-ranked documents with strong contextual alignment to insulin-related conditions and high blood sugar. However, some results included rare syndromes, which may be less directly relevant to the query focus.  
  
**Key Takeaway:**    
Pure semantic search effectively identifies related medical conditions and context-rich information but may require additional filtering or re-ranking to prioritize the most clinically relevant documents.  

In [ ]:
class SemanticSearchStrategy:
    """Pure semantic search using vector similarity."""
    
    def __init__(self, vector_store, name="Semantic Search"):
        self.vector_store = vector_store
        self.name = name
    
    def search(self, query: str, k: int = 5) -> List[Tuple[any, float]]:
        """
        Perform pure semantic search.
        
        Args:
            query (str): Search query
            k (int): Number of results to return
            
        Returns:
            List[Tuple[Document, float]]: Documents with similarity scores
        """
        start_time = time.time()
        
        # Perform similarity search with scores
        results = self.vector_store.similarity_search_with_score(query, k=k)
        
        search_time = time.time() - start_time
        
        return results, search_time
    
    def evaluate_query(self, query: str, k: int = 5) -> Dict:
        """Evaluate a single query and return detailed results."""
        results, search_time = self.search(query, k)
        
        evaluation = {
            'strategy': self.name,
            'query': query,
            'num_results': len(results),
            'search_time': search_time,
            'results': []
        }
        
        for i, (doc, score) in enumerate(results):
            evaluation['results'].append({
                'rank': i + 1,
                'similarity_score': score,
                'document_id': doc.metadata.get('document_id', 'N/A'),
                'content_preview': doc.page_content[:150] + "...",
                'source': doc.metadata.get('document_url', 'N/A')
            })
        
        return evaluation

# Initialize semantic search strategy
semantic_search = SemanticSearchStrategy(vector_store)

# Test with a sample query
sample_query = "What are the early signs and available therapies for high blood sugar caused by lack of insulin?"
evaluation = semantic_search.evaluate_query(sample_query)

print(f"🔍 === Strategy 1: {evaluation['strategy']} ===")
print(f"Query: '{evaluation['query']}'")
print(f"Results found: {evaluation['num_results']}")
print(f"Search time: {evaluation['search_time']:.4f} seconds")

print(f"\n📄 Top Results:")
for result in evaluation['results'][:3]:
    print(f"\n  Rank {result['rank']} (Score: {result['similarity_score']:.4f})")
    print(f"  Content: {result['content_preview']}")
    print(f"  Document ID: {result['document_id']}")

print(f"\n✅ Pure Semantic Search strategy implemented and tested!")

### Strategy 2: Adaptive Threshold Semantic Search  
  
**Purpose:**    
This method enhances pure semantic search by applying a dynamic relevance score threshold. The threshold is automatically calculated from the score distribution for each query, based on a chosen percentile. This ensures that only the top proportion of most relevant results are retained, filtering out lower-quality matches.  
  
**Evaluation Summary:**    
- **Query:** "What are the early signs and available therapies for high blood sugar caused by lack of insulin?"    
- **Approach:** Tested at 70%, 80%, and 90% percentiles (higher percentile = more selective).    
- **Key Results:**    
  - **70% percentile:** Kept 3 out of 10 results (Threshold: 0.4051, Time: 2.11s)    
  - **80% percentile:** Kept 2 out of 10 results (Threshold: 0.4972, Time: 0.53s)    
  - **90% percentile:** Kept 1 out of 10 results (Threshold: 0.7934, Time: 0.45s)    
  
**Observations:**    
- Adaptive filtering drastically reduces noise and focuses on the most relevant matches.    
- Higher percentiles yield fewer but more precise results, at the cost of coverage.    
- Search times were significantly faster than pure semantic search, particularly at higher selectivity levels.  
  
**Key Takeaway:**    
This approach is effective for high-precision retrieval in clinical contexts where irrelevant results could be misleading. The percentile parameter provides flexible control over selectivity, allowing tuning for different use cases.  

In [ ]:
class AdaptiveThresholdSemanticSearch(SemanticSearchStrategy):
    """
    Semantic search with adaptive relevance score threshold filtering.
    Threshold is chosen dynamically from score distribution based on a percentile.
    """
    
    def __init__(self, vector_store, percentile: float = 80, name="Adaptive Threshold Semantic Search"):
        """
        Args:
            vector_store: LangChain-compatible vector store (e.g., Chroma)
            percentile: Percentile cutoff for keeping results (0-100)
                        Example: 80 = keep top 20% most similar results
        """
        super().__init__(vector_store, name)
        self.percentile = percentile
        self.dynamic_threshold = None  # Will be set on first query
    
    def search(self, query: str, k: int = 10) -> Tuple[List[Tuple[any, float]], float]:
        """
        Perform semantic search, compute dynamic threshold, and filter results.
        Returns: (filtered_results, search_time)
        """
        start_time = time.time()
        
        # First get regular similarity search results (without threshold filtering)
        try:
            import warnings
            import sys
            import os
            
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                old_stdout = sys.stdout
                sys.stdout = open(os.devnull, 'w')
                try:
                    # Use similarity_search_with_score to get distance scores first
                    distance_results = self.vector_store.similarity_search_with_score(query, k=k)
                    # Convert distance to relevance scores using a better approach
                    results = []
                    
                    # Extract all distances to normalize them properly
                    distances = [distance for _, distance in distance_results]
                    
                    # Use rank-based scoring or inverse distance scoring
                    if distances:
                        max_distance = max(distances)
                        min_distance = min(distances)
                        distance_range = max_distance - min_distance
                        
                        for doc, distance in distance_results:
                            if distance_range > 0:
                                # Normalize distance to 0-1 range, then invert (lower distance = higher relevance)
                                normalized_distance = (distance - min_distance) / distance_range
                                relevance_score = 1.0 - normalized_distance
                            else:
                                # All distances are the same
                                relevance_score = 1.0
                            results.append((doc, relevance_score))
                    
                finally:
                    sys.stdout.close()
                    sys.stdout = old_stdout
        except Exception as e:
            print(f"⚠️ Search error: {e}")
            results = []
        
        if not results:
            self.dynamic_threshold = None
            return [], time.time() - start_time
        
        # Extract scores for threshold calculation
        scores = [score for _, score in results]
        
        # Compute dynamic threshold based on requested percentile
        # For percentile=80, we keep top 20% (scores above 80th percentile)
        self.dynamic_threshold = float(np.percentile(scores, self.percentile))
        
        # Keep only results above threshold
        filtered_results = [(doc, score) for doc, score in results if score >= self.dynamic_threshold]
        
        return filtered_results, time.time() - start_time
    
    def evaluate_query(self, query: str, k: int = 10) -> Dict:
        """Evaluate query with adaptive threshold filtering details."""
        results, search_time = self.search(query, k)
        
        threshold_str = f"{self.dynamic_threshold:.4f}" if self.dynamic_threshold is not None else "N/A"
        
        evaluation = {
            'strategy': f"{self.name} (percentile={self.percentile}%, threshold={threshold_str})",
            'query': query,
            'initial_k': k,
            'num_results_after_filtering': len(results),
            'search_time': search_time,
            'dynamic_threshold': self.dynamic_threshold,
            'results': []
        }
        
        for i, (doc, score) in enumerate(results):
            evaluation['results'].append({
                'rank': i + 1,
                'relevance_score': score,
                'document_id': doc.metadata.get('document_id', 'N/A'),
                'content_preview': doc.page_content[:150] + "...",
                'source': doc.metadata.get('document_url', 'N/A')
            })
        
        return evaluation
    

# Test adaptive threshold search with different percentiles
percentiles_to_test = [70, 80, 90]

print(f"\n🔍 === Strategy 2b: Adaptive Threshold Semantic Search ===")
print(f"📋 Query: '{sample_query}'")
print("=" * 80)

for percentile in percentiles_to_test:
    adaptive_search = AdaptiveThresholdSemanticSearch(vector_store, percentile=percentile)
    evaluation = adaptive_search.evaluate_query(sample_query)
    
    threshold_display = evaluation['dynamic_threshold']
    if threshold_display is None:
        threshold_display = "N/A"
    else:
        threshold_display = f"{threshold_display:.4f}"
    
    print(f"\n🎯 Percentile: {percentile}% (keeps top {100-percentile}% of results)")
    print(f"   Dynamic threshold: {threshold_display}")
    print(f"   Results kept: {evaluation['num_results_after_filtering']}/{evaluation['initial_k']}")
    print(f"   Search time: {evaluation['search_time']:.4f}s")
    
    if evaluation['results']:
        print("   📑 Top results:")
        for r in evaluation['results'][:2]:  # Show top 2
            print(f"      • Score: {r['relevance_score']:.4f} | Doc ID: {r['document_id']}")
            print(f"        Preview: {r['content_preview'][:80]}...")
    else:
        print("   ❌ No results passed the adaptive filter")

print(f"\n" + "=" * 80)
print(f"✅ Adaptive threshold search completed!")
print(f"💡 Higher percentiles = more selective filtering")
print(f"🎯 Adaptive thresholds adjust based on actual score distribution")

### Strategy 3: Hybrid Search (Semantic + Keyword/BM25)  
  
**Purpose:**    
Hybrid Search combines the strengths of **semantic embeddings** and **keyword-based BM25** retrieval to improve both coverage and precision. Semantic search captures contextual meaning, while BM25 ensures that exact keyword matches are prioritized. A weighted scoring mechanism merges both ranking signals, enabling more balanced retrieval performance.  
  
**Evaluation Summary:**    
- **Query:** "What are the early signs and available therapies for high blood sugar caused by lack of insulin?"    
- **Weights:** Semantic (70%), Keyword/BM25 (30%)    
- **Top Results Comparison:**    
  - **Semantic Search:** Focused on semantically related rare conditions (DocIDs 297, 328, 414)    
  - **BM25 Keyword Search:** Returned the same top two results but with stronger keyword match scores, plus an additional different third result (DocID 711)    
- **Final Hybrid Ranking:**    
  1. **DocID 297** – Score: 1.00    
  2. **DocID 328** – Score: 0.737    
  3. **DocID 414** – Score: 0.231    
  
**Observations:**    
- Hybrid ranking aligned top results from both methods, ensuring that documents relevant both contextually and lexically are prioritized.    
- Retained the same top 2 documents across all methods, but improved ranking confidence by combining scores.    
- Helps mitigate cases where semantic search may include tangential results or BM25 misses conceptually relevant ones.  
  
**Key Takeaway:**    
The hybrid approach offers a **balanced retrieval strategy** that benefits from semantic understanding while maintaining the precision of keyword matching, making it a strong candidate for clinical search tasks where both terminology and context matter.  

In [ ]:

import re  
import time  
import numpy as np  
from rank_bm25 import BM25Okapi  
from typing import List, Tuple, Dict  
  
# --- Built-in English stopwords ---  
ENGLISH_STOPWORDS = {  
    "i","me","my","myself","we","our","ours","ourselves","you","your","yours",  
    "yourself","yourselves","he","him","his","himself","she","her","hers",  
    "herself","it","its","itself","they","them","their","theirs","themselves",  
    "what","which","who","whom","this","that","these","those","am","is","are",  
    "was","were","be","been","being","have","has","had","having","do","does",  
    "did","doing","a","an","the","and","but","if","or","because","as","until",  
    "while","of","at","by","for","with","about","against","between","into",  
    "through","during","before","after","above","below","to","from","up","down",  
    "in","out","on","off","over","under","again","further","then","once","here",  
    "there","when","where","why","how","all","any","both","each","few","more",  
    "most","other","some","such","no","nor","not","only","own","same","so",  
    "than","too","very","s","t","can","will","just","don","should","now"  
}  
  
# --- Simple tokenizer ---  
def simple_tokenize(text: str) -> List[str]:  
    tokens = re.findall(r'\b[a-z]+\b', text.lower())  
    return [t for t in tokens if t not in ENGLISH_STOPWORDS]  
  
# --- Hybrid Search Class ---  
class HybridSearchStrategy:  
    def __init__(self, vector_store, processor, semantic_weight=0.7, keyword_weight=0.3):  
        self.vector_store = vector_store  
        self.processor = processor  
        self.semantic_weight = semantic_weight  
        self.keyword_weight = keyword_weight  
  
        # Prepare documents  
        self.documents = processor.get_langchain_documents()  
        self.doc_texts = [doc.page_content for doc in self.documents]  
        self.doc_id_to_index = {doc.metadata.get('document_id'): i for i, doc in enumerate(self.documents)}  
  
        # Tokenize and build BM25 index  
        tokenized_docs = [simple_tokenize(text) for text in self.doc_texts]  
        self.bm25 = BM25Okapi(tokenized_docs)  
        print(f"📊 BM25 index created for {len(tokenized_docs)} documents (offline mode)")  
  
    def keyword_search(self, query: str, k: int = 10):  
        """BM25 keyword search"""  
        query_tokens = simple_tokenize(query)  
        bm25_scores = self.bm25.get_scores(query_tokens)  
        top_indices = np.argsort(bm25_scores)[::-1][:k]  
        return [(idx, bm25_scores[idx]) for idx in top_indices if bm25_scores[idx] > 0]  
  
    def _normalize_scores(self, score_dict: Dict[int, float]) -> Dict[int, float]:  
        if not score_dict:  
            return {}  
        min_score = min(score_dict.values())  
        max_score = max(score_dict.values())  
        if max_score == min_score:  
            return {k: 0.0 for k in score_dict}  
        return {k: (v - min_score) / (max_score - min_score) for k, v in score_dict.items()}  
  
    def search(self, query: str, k: int = 5):  
        """Hybrid search combining semantic + keyword scores"""  
        start_time = time.time()  
  
        # Semantic search  
        semantic_results = self.vector_store.similarity_search_with_relevance_scores(query, k=k*2)  
        semantic_scores = {}  
        for doc, score in semantic_results:  
            idx = self.doc_id_to_index.get(doc.metadata.get('document_id'))  
            if idx is not None:  
                semantic_scores[idx] = float(score)  
  
        # Keyword search  
        keyword_results = self.keyword_search(query, k=k*2)  
        keyword_scores = {idx: score for idx, score in keyword_results}  
  
        # Normalize both scores to 0–1  
        semantic_scores = self._normalize_scores(semantic_scores)  
        keyword_scores = self._normalize_scores(keyword_scores)  
  
        # Combine weighted scores  
        combined_scores = {}  
        for idx in set(semantic_scores.keys()) | set(keyword_scores.keys()):  
            combined_scores[idx] = (  
                self.semantic_weight * semantic_scores.get(idx, 0.0) +  
                self.keyword_weight * keyword_scores.get(idx, 0.0)  
            )  
  
        # Sort final results  
        sorted_results = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)[:k]  
        final_results = [(self.documents[idx], score) for idx, score in sorted_results]  
  
        return final_results, time.time() - start_time  
  
    def compare_search(self, query: str, k: int = 5):  
      """Show semantic vs keyword results, and final merged hybrid results without double-calling."""  
    
      # ---- Run searches ONCE ----  
      semantic_results = self.vector_store.similarity_search_with_relevance_scores(query, k=k*2)  
      keyword_results = self.keyword_search(query, k=k*2)  
    
      # ---- Convert to dictionaries ----  
      semantic_scores = {}  
      for doc, score in semantic_results:  
          idx = self.doc_id_to_index.get(doc.metadata.get('document_id'))  
          if idx is not None:  
              semantic_scores[idx] = float(score)  
    
      keyword_scores = {idx: score for idx, score in keyword_results}  
    
      # ---- Normalize scores ----  
      semantic_scores = self._normalize_scores(semantic_scores)  
      keyword_scores = self._normalize_scores(keyword_scores)  
    
      # ---- Build top-k lists for display ----  
      semantic_list = [  
          {"rank": rank, "doc_id": doc.metadata.get("document_id", "N/A"), "score": round(float(score), 3)}  
          for rank, (doc, score) in enumerate(semantic_results[:k], start=1)  
      ]  
      keyword_list = [  
          {"rank": rank, "doc_id": self.documents[idx].metadata.get("document_id", "N/A"), "score": round(float(score), 3)}  
          for rank, (idx, score) in enumerate(keyword_results[:k], start=1)  
      ]  
    
      # ---- Print comparison table ----  
      print(f"\n🔍 Query: {query}")  
      print(f"{'Semantic Search':<30} | {'Keyword Search (BM25)':<30}")  
      print("-" * 65)  
      for i in range(max(len(semantic_list), len(keyword_list))):  
          sem = semantic_list[i] if i < len(semantic_list) else {}  
          key = keyword_list[i] if i < len(keyword_list) else {}  
          print(f"{sem.get('rank',''):>2}. DocID {sem.get('doc_id',''):>5} ({sem.get('score','')})"  
                f" | {key.get('rank',''):>2}. DocID {key.get('doc_id',''):>5} ({key.get('score','')})")  
    
      # ---- Merge scores once (Hybrid) ----  
      combined_scores = {}  
      for idx in set(semantic_scores.keys()) | set(keyword_scores.keys()):  
          combined_scores[idx] = (  
              self.semantic_weight * semantic_scores.get(idx, 0.0) +  
              self.keyword_weight * keyword_scores.get(idx, 0.0)  
          )  
    
      merged_results = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)[:k]  
    
      print("\n📊 Final Hybrid Results:")  
      for rank, (idx, score) in enumerate(merged_results, start=1):  
          print(f"{rank}. DocID {self.documents[idx].metadata.get('document_id','N/A')} - Score: {round(score,3)}")  
    
      return semantic_list, keyword_list, merged_results  
      
# =============================  
# 🚀 Initialize Hybrid Search with BM25  
# =============================  
print("🚀 Initializing Hybrid Search with BM25...")  
  
# Create the hybrid search instance  
hybrid = HybridSearchStrategy(vector_store, processor)  
  
query = "What are the early signs and available therapies for high blood sugar caused by lack of insulin?"  
hybrid.compare_search(query, k=3)  


### Strategy 4: LLM-based Re-ranking with Explanations  
  
**Purpose:**    
This strategy uses a **Large Language Model (GPT-4)** to refine search results by re-ranking retrieved chunks according to their direct relevance to the query. While the initial retrieval is done via semantic search, the LLM evaluates the content holistically, prioritizing chunks that most clearly address the user’s question. An explanation is also generated to provide transparency behind the ranking decisions.  
  
**Evaluation Summary:**    
- **Query:** "What are the early signs and available therapies for high blood sugar caused by lack of insulin?"    
- **Process:**    
  1. Retrieve top 10 results via semantic search.    
  2. Present them to GPT-4 with instructions to rank the top 3 by relevance and explain the reasoning.    
  3. Extract the ranking order and explanation for output.  
- **Final LLM Ranking:**    
  1. **Chunk 2 – Wolfram Syndrome:** Directly discusses high blood sugar from insulin deficiency and explicitly mentions insulin replacement therapy.    
  2. **Chunk 1 – Donohue Syndrome:** Details severe insulin resistance and its impact on glucose regulation, indirectly relevant to therapies.    
  3. **Chunk 4 – Glycogen Storage Disease Type VI:** Primarily about low blood sugar (hypoglycemia); less relevant to the question focus.  
  
**Observations:**    
- LLM re-ranking improved precision by placing therapy-relevant content (Wolfram syndrome with insulin therapy) at the top.    
- Provided clear reasoning for each choice, aiding interpretability.    
- Reduced the presence of tangentially related results in the top set.  
  
**Key Takeaway:**    
LLM re-ranking is effective for **precision-focused retrieval**, especially when the goal is to directly answer a specific question. The explanation feature enhances trust and transparency, making it valuable in high-stakes domains like healthcare.  



In [ ]:
def get_response(prompt: str, model: str = "gpt-4") -> str:  
    """  
    Sends a prompt to the OpenAI API and returns the model's text response.  
    """  
    response = chat_client.chat.completions.create(  
        model=model,  
        messages=[  
            {"role": "system", "content": "You are a helpful AI assistant."},  
            {"role": "user", "content": prompt}  
        ],  
        temperature=0  
    )  
    return response.choices[0].message.content.strip()  
  
def llm_rerank_with_explanation(query: str, top_k: int = 3, retrieved_k: int = 10):  
    """  
    Retrieves documents from ChromaDB and reranks them using GPT, with explanations.  
      
    Args:  
        query (str): The user’s input question.  
        top_k (int): Number of top chunks to return after reranking.  
        retrieved_k (int): Number of chunks to initially retrieve from vector DB.  
    """  
    # Step 1: Retrieve docs from vector DB  
    retrieved_docs = vector_store.similarity_search(query, k=retrieved_k)  
  
    # Step 2: Prepare ranking prompt with explanation request  
    prompt = (  
        f"You are helping rank document chunks based on how well they answer this question:\n\n"  
        f"Question: {query}\n\n"  
        "Here are the chunks:\n\n"  
    )  
  
    for i, doc in enumerate(retrieved_docs):  
        prompt += f"Chunk {i+1}:\n{doc.page_content.strip()}\n\n"  
  
    prompt += (  
        f"Please rank the top {top_k} chunks in order of relevance.\n"  
        "First, give the ranking in this exact format:\n"  
        "Ranking: Chunk 3, Chunk 1, Chunk 5\n"  
        "Then on the next lines, explain briefly why you chose that order."  
    )  
  
    # Step 3: Call GPT for reranking with explanation  
    gpt_output = get_response(prompt)  
    print("GPT Rerank + Explanation Output:\n", gpt_output)  
  
    # Step 4: Extract chunk numbers from GPT output  
    ranking_line = next((line for line in gpt_output.split("\n") if line.startswith("Ranking:")), "")  
    chunk_order = [  
        int(s.strip().split()[-1]) - 1  
        for s in ranking_line.replace("Ranking:", "").split(',')  
        if s.strip().startswith("Chunk")  
    ]  
  
    # Step 5: Extract explanation (everything after the ranking line)  
    explanation_lines = []  
    found_ranking = False  
    for line in gpt_output.split("\n"):  
        if found_ranking:  
            explanation_lines.append(line)  
        if line.startswith("Ranking:"):  
            found_ranking = True  
    explanation = "\n".join(explanation_lines).strip()  
  
    # Step 6: Return sorted chunk objects and explanation  
    reranked_docs = [  
        retrieved_docs[i]  
        for i in chunk_order  
        if 0 <= i < len(retrieved_docs)  
    ]  
  
    return reranked_docs, explanation  
  
query = "What are the early signs and available therapies for high blood sugar caused by lack of insulin?"  
top_docs, reasoning = llm_rerank_with_explanation(query, top_k=3)  
  
print("\nTop Ranked Chunks:")  
for idx, d in enumerate(top_docs, 1):  
  print(f"{idx}. {d.page_content}")  
  
print("\nExplanation for ranking:\n", reasoning) 

### Strategy 5: Hybrid Search + LLM-based Re-ranking with Explanations  
  
**Purpose:**    
This strategy combines the **breadth and balance** of Hybrid Search (semantic + BM25 keyword weighting) with the **precision and interpretability** of LLM-based re-ranking. The hybrid retrieval ensures that results capture both contextual meaning and exact keyword matches, while the LLM (GPT-4) reorders them by evaluating their direct relevance to the query and providing a rationale for the ranking.  
  
**Evaluation Summary:**    
- **Query:** "What are the early signs and available therapies for high blood sugar caused by lack of insulin?"    
- **Process:**    
  1. Retrieve top 10 results using Hybrid Search (semantic weight 0.7, keyword weight 0.3).    
  2. Provide results (with hybrid scores) to GPT-4 for re-ranking and reasoning.    
  3. Extract top 3 most relevant chunks based on LLM judgment.    
- **Final LLM Ranking:**    
  1. **Chunk 2 – Wolfram Syndrome:** Directly addresses high blood sugar from insulin deficiency; explicitly mentions insulin replacement therapy.    
  2. **Chunk 1 – Donohue Syndrome:** Describes severe insulin resistance and glucose regulation issues; relevant to early signs but less on therapies.    
  3. **Chunk 3 – Glycogen Storage Disease Type VI:** Discusses blood sugar regulation issues but focuses on hypoglycemia, making it less directly relevant.  
  
**Observations:**    
- Hybrid retrieval ensured that all top-ranked documents were at least partially relevant, reducing the risk of missing key keyword-matching results.    
- LLM re-ranking improved **precision** by prioritizing therapy-specific and directly relevant content.    
- The explanation step increases transparency, which is especially important for clinical or high-stakes contexts.  
  
**Key Takeaway:**    
This combined approach offers **broad recall**, **high precision**, and **explainability**, making it a strong candidate for workflows requiring both comprehensive retrieval and tight alignment with the user’s query intent.  

In [ ]:
def llm_rerank_hybrid_with_explanation(query: str, top_k: int = 3, retrieved_k: int = 10, model: str = "gpt-4"):  
    """  
    Runs hybrid search (semantic + BM25), then reranks the results with GPT, returning top_k chunks + explanation.  
  
    Args:  
        query (str): The user’s input question.  
        top_k (int): Number of top chunks to return after reranking.  
        retrieved_k (int): Number of chunks to initially retrieve from hybrid search.  
        model (str): GPT model name (default: gpt-4).  
    """  
  
    # Step 1: Retrieve docs using HybridSearchStrategy  
    hybrid_results, _ = hybrid.search(query, k=retrieved_k)  # already returns [(Document, score)]  
  
    # Step 2: Prepare ranking prompt with explanation request  
    prompt = (  
        f"You are helping rank document chunks based on how well they answer this question:\n\n"  
        f"Question: {query}\n\n"  
        "Here are the chunks:\n\n"  
    )  
  
    for i, (doc, score) in enumerate(hybrid_results):  
        prompt += f"Chunk {i+1} (Hybrid Score: {round(score,3)}):\n{doc.page_content.strip()}\n\n"  
  
    prompt += (  
        f"Please rank the top {top_k} chunks in order of relevance.\n"  
        "First, give the ranking in this exact format:\n"  
        "Ranking: Chunk 3, Chunk 1, Chunk 5\n"  
        "Then on the next lines, explain briefly why you chose that order."  
    )  
  
    # Step 3: Call GPT for reranking with explanation  
    gpt_output = get_response(prompt, model=model)  
    print("📊 GPT Rerank + Explanation Output:\n", gpt_output)  
  
    # Step 4: Extract chunk numbers from GPT output  
    ranking_line = next((line for line in gpt_output.split("\n") if line.startswith("Ranking:")), "")  
    chunk_order = [  
        int(s.strip().split()[-1]) - 1  
        for s in ranking_line.replace("Ranking:", "").split(',')  
        if s.strip().startswith("Chunk")  
    ]  
  
    # Step 5: Extract explanation (everything after the ranking line)  
    explanation_lines = []  
    found_ranking = False  
    for line in gpt_output.split("\n"):  
        if found_ranking:  
            explanation_lines.append(line)  
        if line.startswith("Ranking:"):  
            found_ranking = True  
    explanation = "\n".join(explanation_lines).strip()  
  
    # Step 6: Return sorted chunk objects and explanation  
    reranked_docs = [  
        hybrid_results[i][0]  # only the Document object  
        for i in chunk_order  
        if 0 <= i < len(hybrid_results)  
    ]  
  
    return reranked_docs, explanation  

query = "What are the early signs and available therapies for high blood sugar caused by lack of insulin?"  
  
top_docs, reasoning = llm_rerank_hybrid_with_explanation(query, top_k=3, retrieved_k=10)  
  
print("\n🏆 Top Ranked Chunks:")  
for idx, d in enumerate(top_docs, 1):  
    print(f"{idx}. {d.page_content[:200]}...")  # truncate for display  
  
print("\n💡 Explanation for ranking:\n", reasoning)  

## 📊 Comparative Summary of Retrieval Strategies (Corrected)  
  
| **Strategy** | **Description** | **Strengths** | **Limitations** | **Top 3 for Query** ("What are the early signs and available therapies for high blood sugar caused by lack of insulin?") | **Suitability** |  
|--------------|----------------|---------------|------------------|----------------------------------------------------------------------------------------------------------------|------------------|  
| **1. Semantic Search** | Retrieves results based on vector similarity of embeddings. | Captures contextual meaning, good for broader queries. | May return tangential matches if semantic similarity is high but exact keyword match is missing. | 1. Donohue Syndrome (DocID 297) <br> 2. Wolfram Syndrome (DocID 328) <br> 3. Glucose-Galactose Malabsorption (DocID 414) | Good for exploratory search when conceptual relevance is key. |  
| **2. Keyword Search (BM25)** | Ranks documents based on exact keyword matches and term frequency. | Strong for precision with exact terms; interpretable scoring. | Misses semantically relevant docs without exact term match. | 1. Donohue Syndrome (DocID 297) <br> 2. Wolfram Syndrome (DocID 328) <br> *(At stricter thresholds, only top 1 or 2 are kept)* | Good when terminology is consistent and exact match is critical. |  
| **3. Hybrid Search (Semantic + BM25)** | Weighted combination of semantic and keyword scores. | Balances contextual relevance with keyword precision; reduces false positives from pure semantic search. | Weight tuning required; still may include partially relevant docs. |  1. Donohue Syndrome <br> 2. Wolfram Syndrome <br> 3. Glycogen Storage Disease VI | Strong default choice for balanced recall & precision. |  
| **4. LLM-based Re-ranking (Semantic Input)** | Retrieves top semantic matches, then reorders with GPT-4 + explanation. | High precision; LLM considers nuance and context; explainable. | Dependent on quality of initial retrieval; higher cost/latency. | 1. Wolfram Syndrome <br> 2. Donohue Syndrome <br> 3. Glycogen Storage Disease VI | Suitable for targeted Q&A where precision and rationale are critical. |  
| **5. Hybrid Search + LLM Re-ranking** | Retrieves via Hybrid Search, then reorders with GPT-4 + explanation. | Combines strong recall from hybrid retrieval with high precision from LLM; maximizes relevance; explainable results. | Highest computational cost; requires both retrieval infra + LLM. | 1. Wolfram Syndrome <br> 2. Donohue Syndrome <br> 3. Glycogen Storage Disease VI | **Best overall choice** for high-stakes, precision-critical retrieval. |  
  
---  
  
## 🏆 Recommended Best Performing Workflow  
  
**Recommendation:** **Strategy 5 – Hybrid Search + LLM Re-ranking with Explanations**  
  
**Why it wins:**  
- **Best Recall + Precision Balance:** Hybrid retrieval ensures no highly relevant document is missed while avoiding purely semantic or keyword-only pitfalls.  
- **Contextual Judgment:** LLM re-ranking filters and prioritizes chunks that directly answer the question, even when multiple are partially relevant.  
- **Transparency:** The explanation step provides clear reasoning, improving trust in the results.  
- **Domain Suitability:** Especially effective in clinical/medical contexts where both terminology accuracy and contextual nuance matter.  
  
**Trade-offs:**    
- Higher cost and latency compared to other strategies, but justified for scenarios where **accuracy outweighs performance constraints**.  
  
---  
  
**Final Takeaway:**    
> For this assessment, **Hybrid + LLM Re-ranking** offers the most robust, interpretable, and contextually accurate retrieval, making it the **ideal choice for production in high-stakes information retrieval systems**.  

### Final: UnifiedClinicalRAG – End-to-End Retrieval, Re-ranking, and Clinical Answer Generation  
  
**Purpose:**    
The `UnifiedClinicalRAG` pipeline is an **integrated retrieval-augmented generation system** designed for clinical question answering. It combines **hybrid document retrieval** (semantic + keyword matching), **LLM-based re-ranking** for relevance, and **controlled answer generation** that is grounded strictly in retrieved evidence. The pipeline enforces strict rules to avoid hallucination, formats the output as structured JSON, and explicitly includes reasoning, cited evidence IDs, and source URLs.  
  
**Workflow Highlights:**  
- **Step 1 – Retrieval:** Uses hybrid search to gather the most relevant chunks from the document corpus.  
- **Step 2 – Re-ranking:** Passes retrieved chunks to an LLM (GPT-4) for ordering by clinical relevance to the query.  
- **Step 3 – Answer Generation:** Produces a concise, evidence-grounded clinical answer without exposing internal chunk IDs or numbering in the narrative.  
- **Step 4 – Transparency:** Returns reasoning, cited chunk IDs, source links, and stated limitations for interpretability.  
- **Step 5 – Latency Tracking:** Captures retrieval, re-ranking, generation, and total processing times for performance monitoring.  
  
**Evaluation on Sample Query:**    
_Query:_ *"What are the early signs and available therapies for high blood sugar caused by lack of insulin?"_   
- **Answer Summary:** Highlighted early signs from Wolfram and Donohue syndromes, with mention of insulin replacement therapy in relevant cases.    
- **Sources Returned:**    
  - https://ghr.nlm.nih.gov/condition/donohue-syndrome/    
  - https://medlineplus.gov/genetics/condition/wolfram-syndrome/    
  
**Key Takeaway:**    
This unified pipeline is well-suited for **precision-critical, high-trust clinical retrieval tasks**, providing structured, explainable outputs. 

In [ ]:
from dataclasses import dataclass, asdict  
import json, re, statistics, time  
from typing import List, Dict, Any, Optional, Tuple  
  
# --- Retrieved Chunk & Response Data Classes ---  
@dataclass  
class RetrievedChunk:  
    chunk_id: int  
    document_id: str  
    source: str  
    content: str  
    score: float  
  
@dataclass  
class GeneratedResponse:  
    query: str  
    status: str  
    answer: str  
    reasoning: str  
    cited_chunks: List[int]  
    sources: List[str]  
    limitations: List[str]  
    retrieval_latency_sec: float  
    rerank_latency_sec: float  
    generation_latency_sec: float  
    total_latency_sec: float  
  
# --- Rerank + Answer Prompts ---  
RERANK_SYSTEM = (  
    "You are a clinical evidence ranking assistant. "  
    "Rank ONLY by usefulness to answer the given question. "  
    "Do not use any external knowledge — base ranking strictly on provided chunks."  
)  
  
ANSWER_SYSTEM = (  
    "You are a clinical answering assistant. "  
    "Generate responses strictly based on the provided evidence excerpts. "  
    "Do NOT use any external knowledge or LLM internal training data. "  
    "If the question is unrelated, mark as out_of_scope. "  
    "If insufficient evidence is provided, mark as insufficient_context. "  
    "If partially supported, mark as partial. "  
    "If fully supported, mark as complete. "  
    "Never hallucinate or speculate — only include facts present in the excerpts."  
)  
  
RERANK_TEMPLATE = """Question: {query}  
Evidence Chunks (structured JSON):  
{chunk_block}  
Return ranking in format:  
Ranking: Chunk i, Chunk j, Chunk k  
"""  
  
ANSWER_TEMPLATE = """Question: {query}  
  
Evidence Excerpts (each with chunk_id, source, and content):  
{chunk_block}  
  
Create a grounded JSON response following this schema:  
{{  
  "status": "complete|partial|insufficient_context|out_of_scope",  
  "answer": "string (natural prose, DO NOT mention 'chunk', 'excerpt id', or numeric evidence IDs)",  
  "reasoning": "concise justification without exposing internal chunk numbering",  
  "cited_chunks": [list of integer chunk_ids from the evidence above that directly support your answer],  
  "sources": [list of unique source URLs from the cited chunks],  
  "limitations": [explicit gaps / uncertainties]  
}}  

**Special rules**:    
- If there are no relevant source excerpts for answering the question, set `"status": "insufficient_context"` and clearly state in `"answer"` that the question cannot be answered using the available sources.    
  
**Important rules for cited_chunks**:  
- ONLY use chunk_id numbers from the Evidence Excerpts above.  
- Output them as integers (e.g., [1, 2]) — not strings.  
- Include ALL chunk_ids that contributed factual information to the answer.  
  
**Rules for the answer**:  
- DO NOT mention chunk numbers in the answer prose.  
- Use only the provided evidence — no external info.  
- Clearly state limitations when data is missing.  
  
Return ONLY the JSON object.  
"""  
  
JSON_BLOCK_RE = re.compile(r"```(?:json)?\s*({[\s\S]*?})\s*```", re.IGNORECASE)  
  
# --- Utility: Extract JSON ---  
def extract_json(text: str) -> Optional[Dict[str, Any]]:  
    match = JSON_BLOCK_RE.search(text)  
    candidate = match.group(1) if match else text.strip()  
    try:  
        return json.loads(candidate)  
    except Exception:  
        return None  
  
# --- Response Generator (Unified) ---  
class UnifiedClinicalRAG:  
    def __init__(self, processor, vector_store, hybrid, chat_client, max_context_chunks: int = 12):  
        self.processor = processor  
        self.vector_store = vector_store  
        self.hybrid = hybrid  
        self.chat_client = chat_client  
        self.max_context_chunks = max_context_chunks  
  
    def _llm(self, system: str, user: str, temperature: float = 0) -> str:  
        resp = self.chat_client.chat.completions.create(  
            model=CHAT_DEPLOYMENT_NAME,  
            messages=[{"role": "system", "content": system}, {"role": "user", "content": user}],  
            temperature=temperature  
        )  
        return resp.choices[0].message.content.strip()  
  
    def hybrid_retrieve(self, query: str, k: int = 12, score_threshold: float = 0.2) -> Tuple[List[RetrievedChunk], float]:  
        t0 = time.time()  
        results, _ = self.hybrid.search(query, k=k)  
        retrieved = []  
        for idx, (doc, score) in enumerate(results):  
            if score_threshold is not None and score < score_threshold:  
                continue  
            retrieved.append(  
                RetrievedChunk(  
                    chunk_id=idx + 1,  
                    document_id=str(doc.metadata.get('document_id', 'N/A')),  
                    source=str(doc.metadata.get('document_url', '')),  
                    content=doc.page_content.strip(),  
                    score=float(score)  
                )  
            )  
        return retrieved, time.time() - t0  
  
    def rerank(self, query: str, chunks: List[RetrievedChunk], top_k: int = 8) -> Tuple[List[RetrievedChunk], float]:  
        if not chunks:  
            return [], 0.0  
        t0 = time.time()  
        chunk_dicts = []  
        for c in chunks[: self.max_context_chunks]:  
            snippet = c.content[:600].replace("\n", " ").strip()  
            chunk_dicts.append({  
                "chunk_id": c.chunk_id,  
                "source": c.source,  
                "content": snippet  
            })  
        chunk_block = json.dumps(chunk_dicts, ensure_ascii=False, indent=2)  
        prompt = RERANK_TEMPLATE.format(query=query, chunk_block=chunk_block)  
        raw = self._llm(RERANK_SYSTEM, prompt, temperature=0)  
        ranking_line = next((ln for ln in raw.split('\n') if ln.lower().startswith('ranking:')), '')  
        order = [int(m.group(1)) for part in ranking_line.replace('Ranking:', '').split(',') if (m := re.search(r'(\d+)', part))]  
        if not order:  
            order = [c.chunk_id for c in chunks]  
        id_map = {c.chunk_id: c for c in chunks}  
        reranked = [id_map[i] for i in order if i in id_map][:top_k]  
        return reranked, time.time() - t0  
  
    def _clean_text(self, text: str) -> str:  
        if not text:  
            return text  
        text = re.sub(r'(?i)chunk\s*\d+[:\-]*', '', text)  
        text = re.sub(r'\s{2,}', ' ', text).strip()  
        return text  
  
    def generate_answer(self, query: str, ranked: List[RetrievedChunk]) -> Tuple[GeneratedResponse, float]:  
        t0 = time.time()  
        if not ranked:  
            empty = GeneratedResponse(  
                query=query,  
                status='insufficient_context',  
                answer='No relevant content was retrieved from the corpus.',  
                reasoning='No evidence excerpts matched.',  
                cited_chunks=[],  
                sources=[],  
                limitations=['No evidence available in indexed corpus'],  
                retrieval_latency_sec=0, rerank_latency_sec=0, generation_latency_sec=0, total_latency_sec=0  
            )  
            return empty, 0.0  
  
        # --- Structured chunk JSON for LLM ---  
        chunk_dicts = []  
        for c in ranked:  
            snippet = c.content[:900].replace("\n", " ").strip()  
            chunk_dicts.append({  
                "chunk_id": c.chunk_id,  
                "source": c.source,  
                "content": snippet  
            })  
        chunk_block = json.dumps(chunk_dicts, ensure_ascii=False, indent=2)  
  
        # --- Build prompt ---  
        prompt = ANSWER_TEMPLATE.format(query=query, chunk_block=chunk_block)  
        print("DEBUG prompt:", prompt)  
        raw = self._llm(ANSWER_SYSTEM, prompt, temperature=0)  
        print("DEBUG raw:", raw)  
  
        # --- Parse JSON output ---  
        parsed = extract_json(raw) or {}  
        status = parsed.get('status', 'insufficient_context')  
  
        print("DEBUG cited_chunks raw:", parsed.get('cited_chunks'))  
  
        # --- Normalize cited_chunks ---  
        raw_cited = parsed.get('cited_chunks', [])  
        cited = []  
        for c in raw_cited:  
            try:  
                cid = int(c)  
                cited.append(cid)  
            except (ValueError, TypeError):  
                continue  
  
        valid_ids = {c.chunk_id for c in ranked}  
        cited = [i for i in cited if i in valid_ids]  
  
        # --- Fallback from sources ---  
        if not cited and parsed.get('sources'):  
            id_map = {c.chunk_id: c for c in ranked}  
            cited = [cid for cid, ch in id_map.items() if ch.source in parsed['sources']]  
  
        # --- Sources ---  
        sources = list({s for s in parsed.get('sources', []) if isinstance(s, str)})  
        if not sources and cited:  
            id_map = {c.chunk_id: c for c in ranked}  
            sources = [id_map[i].source for i in cited if i in id_map and id_map[i].source]  
  
        # --- Clean ---  
        clean_answer = self._clean_text(parsed.get('answer', ''))[:3000]  
        clean_reasoning = self._clean_text(parsed.get('reasoning', ''))[:1200]  
        limitations = parsed.get('limitations', [])  
        if not isinstance(limitations, list):  
            limitations = [str(limitations)]  
  
        resp = GeneratedResponse(  
            query=query,  
            status=status,  
            answer=clean_answer,  
            reasoning=clean_reasoning,  
            cited_chunks=cited,  
            sources=sources,  
            limitations=limitations,  
            retrieval_latency_sec=0.0,  
            rerank_latency_sec=0.0,  
            generation_latency_sec=time.time() - t0,  
            total_latency_sec=0.0  
        )  
        return resp, resp.generation_latency_sec  
  
    def answer(self, query: str, retrieve_k: int = 12, rerank_k: int = 6) -> GeneratedResponse:  
        t0 = time.time()  
        chunks, rlat = self.hybrid_retrieve(query, k=retrieve_k)  
        ranked, rrlat = self.rerank(query, chunks, top_k=rerank_k)  
        answer_obj, glat = self.generate_answer(query, ranked)  
        answer_obj.retrieval_latency_sec = rlat  
        answer_obj.rerank_latency_sec = rrlat  
        answer_obj.generation_latency_sec = glat  
        answer_obj.total_latency_sec = time.time() - t0  
        return answer_obj  
  
# --- Simple Evaluation Harness ---  
def evaluate_queries(pipeline: UnifiedClinicalRAG, queries: List[str], verbose: bool = True) -> Dict[str, Any]:  
    results = []  
    for q in queries:  
        resp = pipeline.answer(q)  
        results.append(asdict(resp))  
        if verbose:  
            print(f"Query: {q}\n  Status: {resp.status} | EvidenceRefs: {len(resp.cited_chunks)} | Sources: {len(resp.sources)} | Total {resp.total_latency_sec:.2f}s\n")  
    statuses = [r['status'] for r in results]  
    metrics = {  
        'counts': {s: statuses.count(s) for s in set(statuses)},  
        'avg_total_latency_s': round(statistics.fmean([r['total_latency_sec'] for r in results]), 3),  
        'avg_cited_chunks': round(statistics.fmean([len(r['cited_chunks']) for r in results]), 2)  
    }  
    return {'metrics': metrics, 'results': results}  
  
print("✅ UnifiedClinicalRAG class updated with structured chunk formatting and improved cited_chunks handling.")  

# Instantiate the unified pipeline (requires prior cells run)
unified_pipeline = UnifiedClinicalRAG(
    processor=processor,
    vector_store=vector_store,
    hybrid=hybrid,
    chat_client=chat_client,
    max_context_chunks=12
)

# Demo single query (pretty summary)
sample_query = "What are the early signs and available therapies for high blood sugar caused by lack of insulin?"
resp = unified_pipeline.answer(sample_query)
print("=== Single Query Result ===")
print(f"Question: {resp.query}")
print(f"Status  : {resp.status}")
print(f"Answer  : {resp.answer[:400]}{'...' if len(resp.answer)>400 else ''}")
print(f"Evidence Items Referenced (IDs): {resp.cited_chunks}")
print(f"Sources ({len(resp.sources)}):")
for s in resp.sources:
    print(f"  - {s}")
print("Latency (s): retrieval={:.2f} rerank={:.2f} gen={:.2f} total={:.2f}".format(
    resp.retrieval_latency_sec, resp.rerank_latency_sec, resp.generation_latency_sec, resp.total_latency_sec
))
if resp.limitations:
    print("Limitations:")
    for l in resp.limitations:
        print(f"  - {l}")

# Batch evaluation example with concise table-style output
# query_batch = [
#     "How is diabetic ketoacidosis initially managed?",
#     "List complications of prolonged untreated high blood sugar.",
#     "Explain beta cell regeneration in type 1 diabetes (if present).",
#     "What vaccines are recommended in this corpus?",
# ]
# summary = evaluate_queries(unified_pipeline, query_batch, verbose=False)

# print("\n=== Batch Evaluation Summary ===")
# print("Queries Run:", len(query_batch))
# print("Status Counts:")
# for k, v in summary['metrics']['counts'].items():
#     print(f"  {k}: {v}")
# print(f"Avg Total Latency (s): {summary['metrics']['avg_total_latency_s']}")
# print(f"Avg Evidence Refs   : {summary['metrics']['avg_cited_chunks']}")

# print("\nPer-Query Snapshot:")
# for r in summary['results']:
#     print(f"- [{r['status']:<18}] {r['query'][:65]}{'...' if len(r['query'])>65 else ''} | refs={len(r['cited_chunks'])} | total={r['total_latency_sec']:.2f}s")